<a href="https://colab.research.google.com/github/schreibe/abmplay2/blob/main/VLgrowthABM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

please make your own copy of this notebook by selecting File-> save a copy

In [ ]:
!pip install agentpy

In [ ]:
#Import helpful packages
import agentpy as ap
import numpy as np
import random

# For plotting
import matplotlib.pyplot as plt

In [ ]:
class Farm(ap.Agent):

    def setup(self):
        #Capital as state variable, values will be filled in from parameter list
        self.K = 0.0

        #strategy (benevolent planner 'BP', ignorant agent/free rider 'FR')
        #option A: values from parameter list
        self.strategy = ''
        #(Initial) Investment Strategy

        #option B:
        # agents have random initial investment strategies
        #self.random = self.model.random
        #self.strategy = self.random.choice(['FR', 'BP'])
        #print(self.strategy)

        #temporary variables
        self.production = 0.0
        self.investment = 0.0
        self.consumption = 0.0

    def produce(self):
        #get parameters
        alpha = self.p.alpha
        eta = self.model.statistician.techPrg[0]

        #production function
        self.production = (self.K**alpha)*((eta)**(1-alpha))

    def decideInvestment(self):
        #get parameters
        rho = self.p.rho
        delta = self.p.delta
        alpha = self.p.alpha
        eta = self.model.statistician.techPrg[0]

        #check strategy and compute investment accordingly
        if self.strategy == 'BP':
            self.investment = 1 / (1 + rho) * (rho * self.production  - (1 - delta) * self.K )
            #print(self.investment) (this print statement was just there to check values when debugging)
            # otherwise, self.strategy == 'FR':
        else:
            self.investment = 1 / (1 + alpha * rho) * (alpha * rho * self.production  - (1 - delta) * self.K )
            #print(self.investment) (same here, just to check)

    def consume(self):
        self.consumption = self.production - self.investment

    def invest(self):
        self.K = (1 - model.p.delta) * self.K + self.investment

        #the following function is not used in the very first model version...
    def updateStrategy(self):
        partner = self.model.agents.random().to_list()[0]
        self.strategy = partner.strategy


In [ ]:
class Statistician(ap.Agent):

    def setup(self):
        self.totalK = sum(model.agents.K)
        #adapt eta to distribution of initial capital
        alpha = self.p.alpha
        value = 0.0
        for i in model.agents.K:
            value += i**alpha
        self.techPrg = self.p.eta0 * (self.totalK**alpha/value)**(1/(1-alpha))
        self.gdp = 0.0

    def aggregateCapital(self):
        self.totalK = sum(model.agents.K)

    def updateTechProg(self):
        self.techPrg = sum(model.agents.K) * self.techPrg/self.totalK
        #print(self.techPrg)

    def computeGDP(self):
        self.gdp = sum(model.agents.production)

In [ ]:
class TestModel(ap.Model):

    def setup(self):
        #agents: farms and statistician
        #list of farms
        self.agents = ap.AgentList(self, self.p.farms, Farm)
        #fill farms' initial capital from input list (parameters)
        self.agents.K = ap.AttrIter(self.p.capitalList)
        # for option A above: agents get strategies from parameter list (if option B, comment out the following line)
        self.agents.strategy = ap.AttrIter(self.p.strategyList)
        #for option B it may be interesting to see which agents got which strategy, if if interest, comment in:
        #print(self.agents.strategy)
        #"list" of one statistician, to make it available to the model
        self.statistician = ap.AgentList(self, 1, Statistician)


    def step(self):

        #farms do their steps
        self.agents.produce()
        self.agents.decideInvestment()
        self.agents.consume()
        self.agents.invest()

        #statistician updates technical progress (with total capital from previous step and sum of new capitals)
        self.statistician.updateTechProg()

        #statistician computes overview
        self.statistician.computeGDP()
        self.statistician.aggregateCapital()

        #not for the very beginning... let agents change strategy
        #self.agents.updateStrategy()


    def update(self):
        self.agents.record("strategy")
        self.agents.record("production")
        self.agents.record("investment")
        self.agents.record("consumption")
      #  self.model.record("K0")
       # self.model.record("tech_init")
        self.agents.record("K")
        self.statistician.record("techPrg")
        self.statistician.record("gdp")
        self.statistician.record("totalK")



In [ ]:
parameters = {'farms':10,
    'capitalList': np.ones(10),
    'strategyList': ['BP', 'BP', 'BP', 'BP', 'BP', 'FR', 'FR', 'FR', 'FR', 'FR'],
    'steps':5,
    'alpha': 0.33,
    'delta': 0.05,
    'rho':0.99,
    'eta0':10
             }

model = TestModel(parameters)
results = model.run()

In [ ]:
results

In [ ]:
results.variables.Farm

In [ ]:
results.variables.Statistician

In [ ]:
data = results.variables.Farm
#ax = data.plot()
steps = parameters.get('steps')
span = steps + 1

farms = parameters.get('farms')
j = 0
while j < steps*farms: #j = steps x agents
    i = 1
    for i in range(1,farms+1): #i = no. agents
        data.K[j:j+span].plot( label = 'agent' + str(i))
        #data.K[j:j+span].legend()
        i += 1
        j += span
plt.xlabel('Time step')

plt.title("capital of agents")
plt.legend()